In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [3]:
# 導入資料

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
# 簡單處理

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 10)


In [5]:
# 這次順便用一些預訓練模型來玩一下
# 但大小差有點多，感覺會有點問題

model = keras.applications.ResNet50(
    include_top=False,
    input_shape=(32, 32, 3),
    classes=10
)

model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
__________________________

In [11]:
# 加入 output layer

inputs = model.input

x = layers.GlobalMaxPooling2D()(model(inputs))
outputs = layers.Dense(10, activation='softmax')(x)


model_pretrained = keras.Model(inputs, outputs)


# 是functional, 所以不能用
# model.add(layers.GlobalMaxPooling2D())
# model.add(layers.Dense(10, activation='softmax'))

model_pretrained.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [17]:
def create_model():
    pre = keras.applications.ResNet50(
        include_top=False,
        input_shape=(32, 32, 3),
        classes=10
    )
    inputs  = pre.input
    x = layers.GlobalMaxPooling2D()(model(inputs))
    outputs = layers.Dense(10, activation='softmax')(x)

    model_pretrained = keras.Model(inputs, outputs)
    model_pretrained.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
    )
    return model_pretrained

In [12]:
# 1. 最基本的epoch checkpoint

model_pretrained.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['acc']
)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint',
    monitor='val_loss',
    verbose=1,
    save_weights_only=False,
)

history = model_pretrained.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=5,
    callbacks=[cp_callback],
    validation_data=(x_test, y_test)
)

Epoch 1/5
391/391 [==============================] - 67s 156ms/step - loss: 1.0703 - acc: 0.6502 - val_loss: 4.7192 - val_acc: 0.1001

Epoch 00001: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 2/5
391/391 [==============================] - 58s 148ms/step - loss: 1.0229 - acc: 0.6711 - val_loss: 1.4164 - val_acc: 0.5576

Epoch 00002: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 3/5
391/391 [==============================] - 58s 148ms/step - loss: 1.1284 - acc: 0.6458 - val_loss: 2.8916 - val_acc: 0.2527

Epoch 00003: saving model to /tmp/checkpoint


...

INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 4/5
391/391 [==============================] - 58s 148ms/step - loss: 1.0753 - acc: 0.6574 - val_loss: 3.1499 - val_acc: 0.5433

Epoch 00004: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 5/5
391/391 [==============================] - 60s 153ms/step - loss: 0.7665 - acc: 0.7385 - val_loss: 0.7908 - val_acc: 0.7319

Epoch 00005: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


In [18]:
# 2. earlystop就不試了
# 3. 以steps當做基準設定checkpoint


NUM_STEPS = 2300
BATCH_SIZE = 128
NUM_CHECKPOINTS = 5     # 同上面，當做對比

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint',
    monotor='val_loss',
    verbose=1,
    save_weights_only=False
)

model_pretrained_2 = create_model()

history_2 = model_pretrained_2.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=NUM_CHECKPOINTS,
    callbacks=[cp_callback],
    validation_data=(x_test, y_test),
    ## 重點!!
    steps_per_epoch=NUM_STEPS // NUM_CHECKPOINTS
)

Epoch 1/5
460/460 [==============================] - 75s 150ms/step - loss: 0.6811 - acc: 0.7710 - val_loss: 0.8176 - val_acc: 0.7332

Epoch 00001: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 2/5
460/460 [==============================] - 68s 147ms/step - loss: 0.4869 - acc: 0.8353 - val_loss: 0.7659 - val_acc: 0.7559

Epoch 00002: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 3/5
460/460 [==============================] - 68s 147ms/step - loss: 1.4690 - acc: 0.5525 - val_loss: 1.5880 - val_acc: 0.4627

Epoch 00003: saving model to /tmp/checkpoint


...

INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 4/5
460/460 [==============================] - 70s 152ms/step - loss: 1.3103 - acc: 0.5749 - val_loss: 1.6981 - val_acc: 0.5778

Epoch 00004: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 5/5
460/460 [==============================] - 20s 42ms/step - loss: 1.1211 - acc: 0.6416 - val_loss: 1.2179 - val_acc: 0.5780

Epoch 00005: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


In [20]:
# 4. 虛擬epoch, 看到一樣的案例總數
# 這邊因為沒有資料，做一個不合理的動作

x_train_new = np.concatenate([x_train, x_test], axis=0)
y_train_new = np.concatenate([y_train, y_test], axis=0)

NUM_TRAINING_EXAMPLES = 50000
STOP_POINT = 4.3        # 虛擬的4.3個EPOCH
TOTAL_TRAINING_EXAMPLES = int(NUM_TRAINING_EXAMPLES * STOP_POINT)
BATCH_SIZE = 128
NUM_CHECKPOINTS = 5

steps_per_epoch = (TOTAL_TRAINING_EXAMPLES // (BATCH_SIZE * NUM_CHECKPOINTS))
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint',
    save_weights_only=False,
    verbose=1
)

model_pretrained_3 = create_model()
history = model_pretrained_3.fit(
    x=x_train_new,
    y=y_train_new,
    epochs=NUM_CHECKPOINTS,
    steps_per_epoch=steps_per_epoch,
    batch_size=BATCH_SIZE,
    callbacks=[cp_callback]
)

Epoch 1/5
335/335 [==============================] - 53s 141ms/step - loss: 1.0464 - acc: 0.6503

Epoch 00001: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 2/5
335/335 [==============================] - 49s 145ms/step - loss: 0.6891 - acc: 0.7671

Epoch 00002: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 3/5
335/335 [==============================] - 47s 141ms/step - loss: 0.5524 - acc: 0.8114

Epoch 00003: saving model to /tmp/checkpoint


...

INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 4/5
335/335 [==============================] - 47s 141ms/step - loss: 0.4608 - acc: 0.8423

Epoch 00004: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets
Epoch 5/5
335/335 [==============================] - 47s 141ms/step - loss: 0.3607 - acc: 0.8767

Epoch 00005: saving model to /tmp/checkpoint


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


In [21]:
# 忘了加validation_data
# 因為訓練資料有data leak, 不意外的結果!

model_pretrained_3.evaluate(x_test, y_test)

313/313 [==============================] - 10s 26ms/step - loss: 0.4842 - acc: 0.8349


[0.48424795269966125, 0.8349000215530396]

> 透過上面的方法，可以使得之後要訓練不用重新開始。其中虛擬epoch，以及以step為基礎各有其好處，無論是新樣本的加入，可以查看效果是否為新樣本問題，或者避免epoch之間太長。